In [12]:
from utils import *
import subprocess

In [40]:
# CA setting

# sat_name = 'ssa2670-141.processed.cnf'
# sat_name = 'mrpp_4x4#4_5.processed.cnf'
# sat_name = 'bf0432-007.processed.cnf'
sat_name = 'bmc-ibm-7.processed.cnf'
# sat_name = 'countbitsrotate016.processed.cnf'

sat_path = f'./dataset/formulas/{sat_name}'
num_vars, num_clauses, sat_instance = read_sat(sat_path)
max_len = max([len(clause) for clause in sat_instance])
# print(max_len)


VIG = nx.Graph()
VIG.add_nodes_from(range(num_vars + 1)[1:])
preprocess_VIG(sat_instance, VIG)  # Build a VIG
part_VIG = community.best_partition(VIG)
num_com = len(set(part_VIG.values()))
mod_VIG = community.modularity(part_VIG, VIG)

temp = open("temp", "w")
subprocess.call([f"/Users/wenweihuang/Workspace/Net2SAT/eval/ca", f"-n {num_vars}", f"-m {num_clauses}", f'-k {3}', f'-c {num_com}', f'-Q {mod_VIG}'], stdout=temp)
temp.close()
temp2 = open('temp2', 'w')
with open("temp", "r") as f:
    for line in f.readlines()[7:]:
        temp2.write(line)
temp2.close()

num_vars, num_clauses, temp_instance = read_sat('./temp2')

features = [
        "clu. VIG",
        "clu. LIG",
        "mod. VIG",
        "mod. LIG",
        "mod. VCG",
        "mod. LCG"
]

metrics = eval_solution(temp_instance, num_vars)
for feature, value in zip(features, metrics):
    print(f'{feature}: {value}')
# f"ca -n {num_vars} -m {num_clauses} -k {max_len} -c {num_com} -Q {mod_VIG}"

clu. VIG: 0.19733825564104762
clu. LIG: 0.12372328866696802
mod. VIG: 0.7041788304984458
mod. LIG: 0.7175855928013319
mod. VCG: 0.7731171119450718
mod. LCG: 0.6219127893738713


In [63]:
# PS setting

sat_name = 'ssa2670-141.processed.cnf'
# sat_name = 'mrpp_4x4#4_5.processed.cnf'
# sat_name = 'bf0432-007.processed.cnf'
# sat_name = 'bmc-ibm-7.processed.cnf'
# sat_name = 'countbitsrotate016.processed.cnf'

sat_path = f'./dataset/formulas/{sat_name}'
num_vars, num_clauses, sat_instance = read_sat(sat_path)
clau_lens = [len(clause) for clause in sat_instance]
min_len = min(clau_lens)
avg_len = sum(clau_lens)/num_clauses
T = 1.5
feats = []
f = open("blah.txt", "w")
subprocess.call(
    [
        "/Users/wenweihuang/Workspace/Net2SAT/eval/scalefree",
        sat_path,
    ],
    stdout=f,
)
f.close()
with open("blah.txt", "r") as f:
    for line in f.readlines():
        if "beta" in line:
            feats.append(line.split()[-1])
print(feats)

temp = open("temp", "w")
subprocess.call([f"/Users/wenweihuang/Workspace/Net2SAT/eval/ps-sat", f"-n {num_vars}", f"-m {num_clauses}", f'-k {avg_len - min_len}', f'-K {min_len}', f'-b {feats[0]}', f'-B {feats[1]}', f'-T {T}'], stdout=temp)
temp.close()
temp2 = open('temp2', 'w')
with open("temp", "r") as f:
    for line in f.readlines()[12:]:
        temp2.write(line)
temp2.close()

num_vars, num_clauses, temp_instance = read_sat('./temp2')

features = [
        "clu. VIG",
        "clu. LIG",
        "mod. VIG",
        "mod. LIG",
        "mod. VCG",
        "mod. LCG"
]

metrics = eval_solution(temp_instance, num_vars)
for feature, value in zip(features, metrics):
    print(f'{feature}: {value}')

['0.264954', '0.783067']
clu. VIG: 0.40557487436023976
clu. LIG: 0.2800705222533292
mod. VIG: 0.1628590785907859
mod. LIG: 0.29853471991607294
mod. VCG: 0.43267970177327164
mod. LCG: 0.4518891679780731


Rigid edges: 7 (truncated) + 747
Flexible edges: 8 (truncated) + 369
